# Accompanying notebook to Optimization Stories in Logistics and Transportation

By Joaquim Gromicho, ORTEC & The Analytics Academy, 2020

In [1]:
def TableSeatSimple( members, capacity, k ):
    from pyomo.environ import ConcreteModel, Var, Objective, Constraint, minimize
    
    model             = ConcreteModel("Dick's seat plan")
    model.nofFamilies = len( members )
    model.nofTables   = len( capacity )
    model.families    = range( model.nofFamilies ) 
    model.tables      = range( model.nofTables ) 
    model.x           = Var( model.families, model.tables, bounds=(0,k) )
    model.goal        = Objective(expr=0,sense= minimize)
    
    def SatisfyTableCapacity( model, t ):
        return sum([model.x[f,t] for f in model.families]) <= capacity[t]
    def SeatAllMembers( model, f ):
        return sum([model.x[f,t] for t in model.tables]) == members[f]

    model.fit = Constraint( model.tables, rule=SatisfyTableCapacity )
    model.sit = Constraint( model.families, rule=SeatAllMembers )
        
    return model

In [2]:
import pyomo.environ as pyo

seatplan = TableSeatSimple( [6,8,2,9,13,1], [8,8,10,4,9], 3 )

from pyomo.opt import SolverFactory
%time results = SolverFactory('glpk').solve(seatplan)
print(results.solver.status, results.solver.termination_condition )

if results.solver.termination_condition == 'optimal':
    import pandas as pd
    sol = pd.DataFrame( index=seatplan.families, columns=seatplan.tables)
    for f in seatplan.families:
        for t in seatplan.tables:
            sol.at[f,t] = pyo.value(seatplan.x[f,t])
    from IPython.display import display
    display(sol)
    print('places at table ', list(sol.sum(axis=0)))
    print('members seated  ', list(sol.sum(axis=1)))

    solver failure.
Wall time: 633 ms
ok optimal


0  1  2  3  4
0  3  0  0  0  3
1  0  2  3  0  3
2  0  0  2  0  0
3  3  3  2  1  0
4  2  3  3  3  2
5  0  0  0  0  1

places at table  [8.0, 8.0, 10.0, 4.0, 9.0]
members seated   [6.0, 8.0, 2.0, 9.0, 13.0, 1.0]


# Minimizing the number of tables used

Now we need binary variables!

In [3]:
def TableSeatCompact( members, capacity, k ):
    from pyomo.environ import ConcreteModel, Var, Objective, Constraint, Binary, minimize
    
    model             = ConcreteModel("Dick's seat plan")
    model.nofFamilies = len( members )
    model.nofTables   = len( capacity )
    model.families    = range( model.nofFamilies ) 
    model.tables      = range( model.nofTables ) 
    model.x           = Var( model.families, model.tables, bounds=(0,k) )
    model.y           = Var( model.tables, within=Binary ) # indicate if table is used
    
    model.goal        = Objective(expr=sum([model.y[t] for t in model.tables]),sense= minimize)
    
    def SatisfyTableCapacity( model, t ):
        return sum([model.x[f,t] for f in model.families]) <= capacity[t]*model.y[t]
    def SeatAllMembers( model, f ):
        return sum([model.x[f,t] for t in model.tables]) == members[f]

    model.fit = Constraint( model.tables, rule=SatisfyTableCapacity )
    model.sit = Constraint( model.families, rule=SeatAllMembers )
        
    return model

In [4]:
import pyomo.environ as pyo

seatplan = TableSeatCompact( [6,8,2,9,13,1], [8,8,10,4,9], 3 )

from pyomo.opt import SolverFactory
%time results = SolverFactory('glpk').solve(seatplan)
print(results.solver.status, results.solver.termination_condition )

if results.solver.termination_condition == 'optimal':
    import pandas as pd
    sol = pd.DataFrame( index=seatplan.families, columns=seatplan.tables)
    for f in seatplan.families:
        for t in seatplan.tables:
            sol.at[f,t] = pyo.value(seatplan.x[f,t])
    from IPython.display import display
    display(sol)
    print('tables used     ', int(round(seatplan.goal.expr())))
    print('places at table ', list(sol.sum(axis=0)))
    print('members seated  ', list(sol.sum(axis=1)))

Wall time: 222 ms
ok optimal


0  1  2  3  4
0  3  0  0  0  3
1  0  2  3  0  3
2  0  0  2  0  0
3  3  3  2  1  0
4  2  3  3  3  2
5  0  0  0  0  1

tables used      5
places at table  [8.0, 8.0, 10.0, 4.0, 9.0]
members seated   [6.0, 8.0, 2.0, 9.0, 13.0, 1.0]
